# Writing Exams

- 이메일 답장하기
- 제시문 내용 요약하기
- 자신의 의견쓰기



In [1]:
import json
from typing import List, Union

from tqdm.notebook import tqdm
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser, CommaSeparatedListOutputParser
from langchain.pydantic_v1 import BaseModel, Field
from langchain.schema import HumanMessage, AIMessage, StrOutputParser
import pandas as pd

C:\Users\msh07\miniconda3\Lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
model = ChatOpenAI(model="gpt-3.5-turbo")

## 이메일 답장하기

### 가상의 이메일 생성하기

In [8]:
#여러 유형의 시험을 프롬프트 엔지니어링 만으로 계속 새로 만들 수 있는 함수 생성.
def build_text_sampling_chain(desc):
    prompt_template = PromptTemplate.from_template(template=desc)
    chain = prompt_template | model | StrOutputParser()
    return chain

In [4]:
email_gen_chain = build_text_sampling_chain(desc="영어 Writing 시험에서 이메일에 답장하기에 등장 할 법한 가상의 영어 이메일 하나 만들어줘. 이름 같은 것들도 가상으로 만들어서 채워줘. 영어로")

In [7]:
email_gen_chain.invoke({})

'Subject: Re: Meeting Schedule\n\nDear Mr. Johnson,\n\nThank you for your email regarding the upcoming meeting schedule. I appreciate the information and I will do my best to attend the meeting on the specified date and time.\n\nIf there are any changes or updates, I would greatly appreciate it if you could keep me informed. I look forward to discussing the agenda items and collaborating with the team.\n\nBest regards,\nEmily Smith'

### 답장 평가하기

In [9]:
def build_eval_chain(instruction, reason_desc, score_desc):
    #1. pydantic class 생성.
    class Evaluation(BaseModel): 
        reason: str = Field(description=reason_desc)
        score: int = Field(description=score_desc)

    #2. JsonOutputParser생성
    parser = JsonOutputParser(pydantic_object=Evaluation)
    format_instructions = parser.get_format_instructions()

    #3. 메시지 프롬프트 생성
    human_prompt_template = HumanMessagePromptTemplate.from_template(
                                "# Instruction: {instruction}\n" #instruction = 평가처리 방식
                                "# Context: {context}\n"         #문제
                                "# User: {input}\n"              #사용자의 응답
                                "{format_instructions}",
                                partial_variables={"instruction": instruction,
                                                   "format_instructions": format_instructions})
    
    prompt = ChatPromptTemplate.from_messages(
        [
            human_prompt_template,
        ])
    eval_chain = prompt | model | parser
    return eval_chain

In [19]:
email_eval_chain = build_eval_chain(instruction="User의 응답이 Context의 이메일에 대해 Context의 문맥과 상황에 적절한 표현을 사용했는지 파악하여 적절한 응답인지 추론하고 평가하라",
                                    reason_desc="User의 응답이 Context의 이메일에 대해 Context의 문맥과 상황에 적절한 표현을 사용했는지 파악하여 적절한 응답인지에 대한 추론",
                                    score_desc="User의 응답이 Context의 이메일에 대해 Context의 문맥과 상황에 적절한 표현을 사용했는지 파악하여 적절한 응답인지에 대한 점수, 0~10점")

In [20]:
email = email_gen_chain.invoke({})

In [21]:
email

'Subject: Invitation to Collaboration\n\nDear Professor Johnson,\n\nI hope this email finds you well. I am writing to express my interest in the opportunity to collaborate with you on a research project. Your expertise in the field of economics has been a source of inspiration for me, and I believe that our combined efforts could lead to valuable insights and contributions to the academic community.\n\nI would be honored to work alongside you and contribute to the advancement of knowledge in our field. Please let me know if you would be open to discussing this further and exploring potential research topics together.\n\nThank you for considering my proposal. I look forward to hearing from you soon.\n\nBest regards,\nSarah Williams'

In [22]:
user_answer = "Dear Emily Johnson. Hi, I'm happy to hear that you are willing to discuss about your job position. I'll be available on Saturday November at 8am. Please come to the Cafe Demel to have the discussion."

In [23]:
eval_result = email_eval_chain.invoke({"context": email, "input": user_answer})

In [24]:
eval_result

{'reason': "User's response is not appropriate as it shows a misunderstanding of the context. The email was about a research collaboration opportunity, not a job position discussion.",
 'score': 3}

## 제시문 내용 요약하기

### 무작위 글 생성

In [25]:
#제시문 만드는 체인 생성.
text_gen_chain = build_text_sampling_chain(desc="영어 Writing 시험에서 단락 요약하기에 등장 할 법한 가상의 영어 단락 하나 만들어줘. 이름 같은 것들도 가상으로 만들어서 채워줘. 영어로")

In [26]:
text = text_gen_chain.invoke({})
text

'As a university student majoring in psychology, Emily is fascinated by the complexities of the human mind. Her latest research project delves into the effects of childhood trauma on adult mental health. Through interviews and surveys, she hopes to gain a better understanding of how past experiences shape our present selves. With the support of her professor Dr. Johnson, Emily is determined to uncover new insights that could potentially revolutionize the field of psychology.'

### 요약 평가하기

In [27]:
summarization_eval_chain = build_eval_chain(instruction="User의 응답이 Context에 대해 문맥과 상황에 적절한 표현을 사용했는지 파악하여 적절한 요약인지 추론하고 평가하라",
                                            reason_desc="User의 응답이 Context에 대해 Context의 문맥과 상황에 적절한 표현을 사용했는지 파악하여 적절한 요약인지에 대한 추론",
                                            score_desc="User의 응답이 Context에 대해 Context의 문맥과 상황에 적절한 표현을 사용했는지 파악하여 적절한 요약인지에 대한 점수, 0~10점")

In [28]:
user_answer = "The innovative advancement and increasing adoption of solar power is emerging as a green energy solution for a sustainable future."

In [29]:
summarization_eval_chain.invoke({"context": text,
                                 "input": user_answer})

{'reason': "User's response is not a proper summary of the given context. It discusses a different topic unrelated to psychology or research.",
 'score': 2}

## 자신의 의견쓰기

### 무작위 이슈 생성


In [36]:
issue_gen_chain = build_text_sampling_chain(desc="영어 Writing 시험에서 자신의 의견쓰기에 등장 할 법한 무작위 이슈 영어 단락 하나 만들어줘. 이름 같은 것들도 가상으로 만들어서 채워줘. 영어로")

In [37]:
issue = issue_gen_chain.invoke({})
issue

'One controversial issue that frequently arises in the modern world is the use of social media by children and teenagers. Many argue that excessive use of platforms such as Instagram, TikTok, and Snapchat can have negative effects on mental health, self-esteem, and social relationships. On the other hand, some believe that these platforms can promote creativity, communication, and social connection among young users. In my opinion, while social media can have its drawbacks, it ultimately comes down to how it is used and supervised by parents and guardians.'

### 의견 평가하기

In [43]:
opinion_eval_chain = build_eval_chain(instruction="User의 응답이 Context에 대해 문맥과 상황에 적절한 표현을 사용했는지 파악하여 요지에 적절한 의견을 주장했는지 평가하라.",
                                      reason_desc="User의 응답이 Context에 대해 문맥과 상황에 적절한 표현을 사용했는지 파악하여 적절한 의견 주장인지 대한 추론",
                                      score_desc="User의 응답이 Context에 대해 문맥과 상황에 적절한 표현을 사용했는지 파악하여 적절한 의견 주장인지에 대한 점수, 0~10점")

In [44]:
user_answer = "The problem of being exposed on social media is the impact for young people's perception can be distorted and lead to social issue. Also social media distrrupt young people's brain development when it comes to morality."

In [45]:
opinion_eval_chain.invoke({"context": issue,
                           "input": user_answer})

{'reason': "The user provided a relevant opinion by mentioning the negative impact of social media on young people's perception and brain development in terms of morality.",
 'score': 8}